# Análise exploratória da RMB (Água, Saneamento e Saúde)
Este notebook é o ponto de partida do nosso TCC em IA para Sustentabilidade. Investigamos como infraestrutura de água/esgoto, qualidade dos serviços, clima e financiamento público influenciam as internações por agravos hídricos nos oito municípios da Região Metropolitana de Belém (2018‑2025). A partir das camadas Silver/Gold, consolidamos evidências para responder às cinco perguntas de negócio apresentadas para a banca: (1) quais ações reduzem mais as internações, (2) qual o retorno por município/sistema, (3) como a chuva modula o risco, (4) qual ranking de priorização para os próximos 12 meses e (5) qual o status frente às metas ODS 6/3/11. As seções seguintes explicam cada etapa antes da execução e geram os arquivos que alimentam diretamente o dashboard no Looker Studio.

## 1. Configuração do ambiente e paths
Configuramos dinamicamente o diretório do projeto (compatível com ambiente local e Google Colab) e definimos as pastas `data/`, `silver/`, `gold/` e o destino dos exports do dashboard. Essa etapa garante que todas as leituras/escritas usem caminhos relativos e evita alterações manuais nos colegas.

In [21]:
import os
from pathlib import Path

try:
    from google.colab import drive  # type: ignore
    IN_COLAB = True
except ModuleNotFoundError:
    drive = None
    IN_COLAB = False

def resolve_repo_root() -> Path:
    env_root = os.getenv("REPO_ROOT") or os.getenv("DATA_ROOT")
    if env_root:
        candidate = Path(env_root).expanduser().resolve()
        if (candidate / "data").exists():
            return candidate
    for candidate in [Path.cwd().resolve()] + list(Path.cwd().resolve().parents):
        if (candidate / "data").exists() and (candidate / "README.md").exists():
            return candidate
    if IN_COLAB:
        if drive is not None:
            drive.mount("/content/drive", force_remount=False)
        colab_repo = Path("/content/drive/MyDrive/projeto-final-curso-i2a2")
        if (colab_repo / "data").exists():
            return colab_repo
    raise FileNotFoundError(
        "Não foi possível localizar o diretório raiz do projeto. Defina a variável de ambiente "
        "REPO_ROOT ou DATA_ROOT apontando para a pasta que contém 'data/'."
    )

PROJECT_ROOT = resolve_repo_root()
DATA_DIR = PROJECT_ROOT / "data"
SILVER_DIR = DATA_DIR / "silver"
GOLD_DIR = DATA_DIR / "gold"
DASHBOARD_EXPORT_DIR = PROJECT_ROOT / "dashboard" / "material_para_dashboard"
DASHBOARD_EXPORT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Projeto raiz: {PROJECT_ROOT}")
print(f"Dados silver: {SILVER_DIR}")
print(f"Dados gold: {GOLD_DIR}")
print(f"Exports dashboard: {DASHBOARD_EXPORT_DIR}")


Projeto raiz: /home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2
Dados silver: /home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2/data/silver
Dados gold: /home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2/data/gold
Exports dashboard: /home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2/dashboard/material_para_dashboard


## 2. Snapshot da base Gold anual
Antes de avançar, inspecionamos as primeiras linhas de `gold_features_ano.parquet` para validar schema, contagem de registros (8 municípios × 8 anos) e checar rapidamente se as colunas esperadas para o dashboard estão presentes.

In [15]:
import pandas as pd

# Visualizar amostra do dataset gold (base oficial para dashboards)
gold_sample_path = GOLD_DIR / "gold_features_ano.parquet"

gold_df = pd.read_parquet(gold_sample_path)
print(f"Registros em gold_features_ano: {gold_df.shape[0]} linhas")
gold_df.head(10)

Registros em gold_features_ano: 64 linhas


,cod_mun,municipio,ano,populacao,idx_atend_agua_total,idx_atend_agua_urbano,idx_coleta_esgoto,idx_tratamento_esgoto,idx_hidrometracao,idx_perdas_distribuicao,...,internacoes_total,internacoes_hidricas,dias_perm_total,dias_perm_hidricas,valor_total,valor_hidricas,internacoes_total_10k,internacoes_hidricas_10k,pct_internacoes_hidricas,valor_medio_internacao
0,1500800,ANANINDEUA,2018,525566.00,NaN,NaN,NaN,NaN,NaN,NaN,...,33625,1907,145459.00,7344.00,33358325.69,687622.26,639.79,36.28,5.67,992.07
1,1500800,ANANINDEUA,2019,530598.00,NaN,NaN,NaN,NaN,NaN,NaN,...,31742,1869,134367.00,6505.00,30968523.41,672328.37,598.23,35.22,5.89,975.63
2,1500800,ANANINDEUA,2020,535547.00,NaN,NaN,NaN,0.18,NaN,NaN,...,28648,1219,124735.00,3632.00,35739684.11,470035.90,534.93,22.76,4.26,1247.55
3,1500800,ANANINDEUA,2021,540410.00,NaN,NaN,NaN,37.44,NaN,NaN,...,24784,213,118372.00,1008.00,39538225.46,252334.72,458.61,3.94,0.86,1595.31
4,1500800,ANANINDEUA,2022,NaN,NaN,NaN,NaN,37.53,NaN,NaN,...,23719,88,118569.00,567.00,39168060.98,128402.19,NaN,NaN,0.37,1651.34
5,1500800,ANANINDEUA,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25715,123,123620.00,645.00,46037252.76,128827.93,NaN,NaN,0.48,1790.29
6,1500800,ANANINDEUA,2024,507838.00,NaN,NaN,NaN,NaN,NaN,NaN,...,26705,155,128119.00,947.00,48798792.23,159256.10,525.86,3.05,0.58,1827.33
7,1500800,ANANINDEUA,2025,509227.00,NaN,NaN,NaN,NaN,NaN,NaN,...,18228,113,89667.00,766.00,34468540.33,165473.29,357.95,2.22,0.62,1890.97
8,1501303,BARCARENA,2018,122294.00,NaN,NaN,NaN,NaN,NaN,NaN,...,7325,111,33445.00,522.00,6404199.63,39106.56,598.97,9.08,1.52,874.29
9,1501303,BARCARENA,2019,124680.00,NaN,NaN,NaN,NaN,NaN,NaN,...,7103,90,32104.00,459.00,6135981.00,33260.20,569.70,7.22,1.27,863.86


## 3. Conferência da população Silver
Mantemos uma amostra controlada de `data/silver/ibge_populacao/populacao.parquet` para garantir que o mapeamento de municípios (códigos IBGE e nomes normalizados) esteja consistente ao longo do notebook e com o arquivo `config/rmb_municipios.csv`.

In [16]:
import pandas as pd
from IPython.display import display

# Amostra controlada da base silver de população (garante compatibilidade local/Colab)
pop_path = SILVER_DIR / "ibge_populacao" / "populacao.parquet"
if not pop_path.exists():
    raise FileNotFoundError(f"Arquivo não encontrado: {pop_path}")

pop = pd.read_parquet(pop_path).copy()
pop["cod_mun"] = pop["cod_mun"].astype(str).str.zfill(7)

catalog_path = PROJECT_ROOT / "config" / "rmb_municipios.csv"
if not catalog_path.exists():
    raise FileNotFoundError("Arquivo de referência config/rmb_municipios.csv não encontrado.")

municipio_lookup = (
    pd.read_csv(catalog_path)
    .assign(
        cod_mun=lambda df: df["ibge_code"].astype(str).str.zfill(7),
        municipio=lambda df: df["name"].str.upper(),
    )[["cod_mun", "municipio"]]
    .drop_duplicates()
 )

municipios_unicos = (
    pop[["cod_mun"]]
    .drop_duplicates()
    .merge(municipio_lookup, on="cod_mun", how="left")
    .sort_values(["cod_mun", "municipio"])
    .reset_index(drop=True)
 )

print(f"Total de municípios únicos (silver população): {municipios_unicos.shape[0]}")
display(municipios_unicos)

Total de municípios únicos (silver população): 8


,cod_mun,municipio
0,1500800,ANANINDEUA
1,1501303,BARCARENA
2,1501402,BELÉM
3,1501501,BENEVIDES
4,1502400,CASTANHAL
5,1504422,MARITUBA
6,1506351,SANTA BÁRBARA DO PARÁ
7,1506500,SANTA IZABEL DO PARÁ


## 4. Preparação do dataset analítico e checagem de cobertura
Nesta etapa alinhamos os seis municípios da RMB aos códigos IBGE, unimos `gold_features_ano` com indicadores do SNIS (fallback quando houver `NaN`) e verificamos a cobertura 2018–2025. Essa validação garante que nenhuma combinação município/ano ficará de fora das análises ou do dashboard.

In [17]:
import pandas as pd
import numpy as np
from IPython.display import display

pd.options.display.float_format = "{:.2f}".format
EXPECTED_YEARS = list(range(2018, 2026))

RMB_CFG = (
    pd.read_csv(PROJECT_ROOT / "config" / "rmb_municipios.csv")
    .query("is_rmb == 1")
    .assign(
        cod_mun=lambda df: df["ibge_code"].astype(str).str.zfill(7),
        municipio_cfg=lambda df: df["name"].str.upper(),
    )
)

print(f"Municípios monitorados (RMB): {RMB_CFG['municipio_cfg'].tolist()}")

GOLD_PATH = GOLD_DIR / "gold_features_ano.parquet"
if not GOLD_PATH.exists():
    raise FileNotFoundError(f"gold_features_ano não encontrado em {GOLD_PATH}")

gold = pd.read_parquet(GOLD_PATH).copy()
gold["cod_mun"] = gold["cod_mun"].astype(str).str.zfill(7)
gold["municipio"] = gold["municipio"].str.upper()

snis_path = GOLD_DIR / "snis_rmb_indicadores_v2.parquet"
if snis_path.exists():
    snis = pd.read_parquet(snis_path).copy()
    snis["cod_mun"] = snis["cod_mun"].astype(str).str.zfill(7)
else:
    snis = pd.DataFrame()
    print("⚠️ Arquivo snis_rmb_indicadores_v2 não encontrado em data/gold. Pulei a etapa de fallback de indicadores de serviço.")

service_fallback_cols = [
    "idx_atend_agua_total",
    "idx_atend_agua_urbano",
    "idx_coleta_esgoto",
    "idx_tratamento_esgoto",
    "idx_hidrometracao",
    "idx_perdas_distribuicao",
    "idx_perdas_lineares",
    "idx_perdas_por_ligacao",
    "tarifa_media_agua",
]

if not snis.empty:
    available_cols = [col for col in service_fallback_cols if col in snis.columns]
    snis_subset = snis[["cod_mun", "ano"] + available_cols]
    gold = gold.merge(snis_subset, on=["cod_mun", "ano"], how="left", suffixes=("", "_snis"))
    for col in available_cols:
        snis_col = f"{col}_snis"
        if snis_col in gold.columns:
            gold[col] = gold[col].fillna(gold[snis_col])
            gold.drop(columns=snis_col, inplace=True)

qualidade_path = GOLD_DIR / "gold_qualidade_agua.parquet"
if not qualidade_path.exists():
    print("ℹ️ Dataset gold_qualidade_agua.* não foi encontrado. Usarei apenas as colunas de conformidade já presentes em gold_features_ano.")

print(f"Registros em gold_features_ano (após merge SNIS): {gold.shape[0]}")
print(f"Colunas disponíveis: {len(gold.columns)}")

coverage = (
    gold.groupby("ano")["cod_mun"]
    .nunique()
    .reset_index(name="municipios_com_dado")
    .sort_values("ano")
)
coverage["municipios_esperados"] = RMB_CFG.shape[0]
coverage["percentual_cobertura"] = coverage["municipios_com_dado"] / coverage["municipios_esperados"]

print("\nCobertura por ano (municípios com dado em relação ao esperado):")
display(coverage)

expected_pairs = pd.MultiIndex.from_product(
    [RMB_CFG["cod_mun"].unique(), EXPECTED_YEARS], names=["cod_mun", "ano"]
)
observed_pairs = pd.MultiIndex.from_frame(gold[["cod_mun", "ano"]].drop_duplicates())
missing_pairs = expected_pairs.difference(observed_pairs)

if len(missing_pairs) == 0:
    print("\nTodos os pares município/ano entre 2018 e 2025 possuem registro em Gold.")
else:
    missing_df = (
        pd.DataFrame(list(missing_pairs), columns=["cod_mun", "ano"])
        .merge(RMB_CFG[["cod_mun", "municipio_cfg"]], on="cod_mun", how="left")
        .sort_values(["ano", "municipio_cfg"])
        .reset_index(drop=True)
    )
    print(f"\nPares município/ano sem dado em Gold ({len(missing_df)} no total):")
    display(missing_df)


Municípios monitorados (RMB): ['BELÉM', 'ANANINDEUA', 'MARITUBA', 'BENEVIDES', 'SANTA BÁRBARA DO PARÁ', 'SANTA IZABEL DO PARÁ', 'CASTANHAL', 'BARCARENA']
ℹ️ Dataset gold_qualidade_agua.* não foi encontrado. Usarei apenas as colunas de conformidade já presentes em gold_features_ano.
Registros em gold_features_ano (após merge SNIS): 64
Colunas disponíveis: 56

Cobertura por ano (municípios com dado em relação ao esperado):


,ano,municipios_com_dado,municipios_esperados,percentual_cobertura
0,2018,8,8,1.00
1,2019,8,8,1.00
2,2020,8,8,1.00
3,2021,8,8,1.00
4,2022,8,8,1.00
5,2023,8,8,1.00
6,2024,8,8,1.00
7,2025,8,8,1.00



Todos os pares município/ano entre 2018 e 2025 possuem registro em Gold.


## 5. Métricas para perguntas de negócio e KPIs do dashboard
Construímos `analysis_df` preenchendo lacunas, normalizando percentuais e derivando variáveis de ação (déficits, perdas, chuva defasada). A seguir calculamos drivers de correlação, ranking de prioridade, retorno por município e o `ods_tracker` com metas (taxa ≤ 30 por 10k hab., cobertura ≥ 95%, tratamento ≥ 80%). Essa célula responde às perguntas 1–5 do roteiro e sustenta os cards do Looker.

In [18]:
import numpy as np

analysis_df = gold.copy().reset_index(drop=True)
analysis_df = analysis_df.merge(
    RMB_CFG[["cod_mun", "municipio_cfg"]], on="cod_mun", how="left"
)
analysis_df["municipio"] = analysis_df["municipio_cfg"].fillna(analysis_df["municipio"]).str.upper()
analysis_df.drop(columns=["municipio_cfg"], inplace=True)
analysis_df["ano"] = analysis_df["ano"].astype(int)
analysis_df.sort_values(["cod_mun", "ano"], inplace=True)
analysis_df.reset_index(drop=True, inplace=True)

columns_to_fill = [
    "idx_atend_agua_total",
    "idx_tratamento_esgoto",
    "idx_coleta_esgoto",
    "idx_perdas_distribuicao",
    "pct_conformes_global",
    "despesa_saude_pc",
    "pct_despesa_investimentos_saude",
    "chuva_total_mm",
    "temp_media_c",
    "populacao",
    "internacoes_hidricas_10k",
    "internacoes_total_10k",
]
for col in columns_to_fill:
    if col in analysis_df.columns:
        analysis_df[col] = analysis_df.groupby("cod_mun")[col].transform(lambda s: s.fillna(s.median()))
        analysis_df[col] = analysis_df[col].fillna(analysis_df[col].median())

percent_cols = [
    "idx_atend_agua_total",
    "idx_tratamento_esgoto",
    "idx_coleta_esgoto",
    "pct_conformes_global",
]
for col in percent_cols:
    if col in analysis_df.columns:
        analysis_df[col] = analysis_df[col].clip(lower=0, upper=100)

analysis_df["deficit_atendimento"] = (100 - analysis_df["idx_atend_agua_total"]).clip(lower=0, upper=100)
analysis_df["deficit_tratamento"] = (100 - analysis_df["idx_tratamento_esgoto"]).clip(lower=0, upper=100)
analysis_df["alerta_qualidade"] = (100 - analysis_df["pct_conformes_global"]).clip(lower=0, upper=100)
analysis_df["perdas_excesso"] = analysis_df["idx_perdas_distribuicao"].clip(lower=0)
analysis_df["chuva_total_mm_lag1"] = analysis_df.groupby("cod_mun")["chuva_total_mm"].shift(1)


def minmax_norm(series: pd.Series) -> pd.Series:
    series = series.fillna(series.median())
    delta = series.max() - series.min()
    if delta == 0 or np.isclose(delta, 0):
        return pd.Series(0.0, index=series.index)
    return (series - series.min()) / delta

analysis_df["score_priorizacao"] = (
    0.40 * minmax_norm(analysis_df["internacoes_hidricas_10k"])
    + 0.20 * minmax_norm(analysis_df["deficit_tratamento"])
    + 0.15 * minmax_norm(analysis_df["alerta_qualidade"])
    + 0.15 * minmax_norm(analysis_df["deficit_atendimento"])
    + 0.10 * minmax_norm(analysis_df["perdas_excesso"])
)
analysis_df["prioridade_categoria"] = pd.cut(
    analysis_df["score_priorizacao"],
    bins=[-np.inf, 0.33, 0.66, np.inf],
    labels=["Estável", "Atenção", "Crítico"],
)

print("Principais drivers das internações hídricas (correlação):")
driver_cols = [
    "idx_atend_agua_total",
    "idx_tratamento_esgoto",
    "idx_coleta_esgoto",
    "idx_perdas_distribuicao",
    "pct_conformes_global",
    "chuva_total_mm",
    "despesa_saude_pc",
    "pct_despesa_investimentos_saude",
]
available_drivers = [col for col in driver_cols if col in analysis_df.columns]
drivers_corr = (
    analysis_df[["internacoes_hidricas_10k"] + available_drivers]
    .corr()
    .loc[available_drivers, "internacoes_hidricas_10k"]
    .to_frame(name="correlacao_com_taxa")
    .sort_values("correlacao_com_taxa", key=lambda s: s.abs(), ascending=False)
)
display(drivers_corr)

lag_corr = analysis_df[["internacoes_hidricas_10k", "chuva_total_mm_lag1"]].dropna().corr().iloc[0, 1]
print(f"Correlação chuva (lag de 1 ano) x taxa de internações: {lag_corr:.3f}")

ano_recente = int(analysis_df["ano"].max())
ranking_cols = [
    "municipio",
    "ano",
    "internacoes_hidricas_10k",
    "idx_atend_agua_total",
    "idx_tratamento_esgoto",
    "pct_conformes_global",
    "perdas_excesso",
    "score_priorizacao",
    "prioridade_categoria",
]
ranking_atual = (
    analysis_df.loc[analysis_df["ano"] == ano_recente, ranking_cols]
    .sort_values("score_priorizacao", ascending=False)
    .reset_index(drop=True)
)
print(f"\nRanking de prioridade para {ano_recente} (Painel Prioridade):")
display(ranking_atual)


def resumo_municipio(grp: pd.DataFrame) -> pd.Series:
    return pd.Series(
        {
            "taxa_media": grp["internacoes_hidricas_10k"].mean(),
            "investimento_medio": grp["pct_despesa_investimentos_saude"].mean(),
            "cobertura_agua_media": grp["idx_atend_agua_total"].mean(),
            "tratamento_esgoto_medio": grp["idx_tratamento_esgoto"].mean(),
            "elasticidade_inv_taxa": grp["pct_despesa_investimentos_saude"].corr(grp["internacoes_hidricas_10k"]),
        }
    )

retorno_municipal = (
    analysis_df.groupby("municipio", group_keys=False)
    .apply(resumo_municipio, include_groups=False)
    .reset_index()
    .sort_values("taxa_media", ascending=False)
)
print("\nRetorno médio por município (quanto investir afeta a taxa):")
display(retorno_municipal)

ODS_TARGETS = {
    "taxa_hidricas_10k_rmb": ("max", 30),
    "pct_conformes_global_media": ("min", 95),
    "idx_atend_agua_total_media": ("min", 95),
    "idx_tratamento_esgoto_media": ("min", 80),
}

ods_tracker = (
    analysis_df.groupby("ano").agg(
        populacao_total=("populacao", "sum"),
        internacoes_hidricas_total=("internacoes_hidricas", "sum"),
        internacoes_total=("internacoes_total", "sum"),
        idx_atend_agua_total_media=("idx_atend_agua_total", "mean"),
        idx_tratamento_esgoto_media=("idx_tratamento_esgoto", "mean"),
        pct_conformes_global_media=("pct_conformes_global", "mean"),
        pct_despesa_investimentos_media=("pct_despesa_investimentos_saude", "mean"),
        chuva_total_mm_media=("chuva_total_mm", "mean"),
    ).reset_index()
)
ods_tracker["taxa_hidricas_10k_rmb"] = (
    ods_tracker["internacoes_hidricas_total"] / ods_tracker["populacao_total"]
) * 10000


def indicator_status(value: float, goal: float, goal_type: str) -> str:
    if pd.isna(value):
        return "Sem dado"
    if goal_type == "max":
        return "OK" if value <= goal else "Alerta"
    return "OK" if value >= goal else "Alerta"

for col, (goal_type, goal_value) in ODS_TARGETS.items():
    ods_tracker[f"status_{col}"] = ods_tracker[col].apply(
        lambda val, gt=goal_value, gtype=goal_type: indicator_status(val, gt, gtype)
    )

print("\nODS Tracker consolidado (RMB):")
display(ods_tracker)


Principais drivers das internações hídricas (correlação):


/home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/an

,correlacao_com_taxa
chuva_total_mm,0.41
pct_despesa_investimentos_saude,-0.34
idx_tratamento_esgoto,0.34
pct_conformes_global,-0.30
despesa_saude_pc,-0.29
idx_atend_agua_total,0.28
idx_perdas_distribuicao,0.19
idx_coleta_esgoto,-0.05


Correlação chuva (lag de 1 ano) x taxa de internações: 0.349

Ranking de prioridade para 2025 (Painel Prioridade):


,municipio,ano,internacoes_hidricas_10k,idx_atend_agua_total,idx_tratamento_esgoto,pct_conformes_global,perdas_excesso,score_priorizacao,prioridade_categoria
0,BARCARENA,2025,4.39,80.72,0.09,67.73,34.02,0.47,Atenção
1,CASTANHAL,2025,6.22,80.72,0.09,75.82,34.02,0.47,Atenção
2,BELÉM,2025,4.33,80.72,0.09,74.46,34.02,0.45,Atenção
3,SANTA IZABEL DO PARÁ,2025,6.57,86.55,0.09,75.82,34.02,0.45,Atenção
4,MARITUBA,2025,1.00,80.72,0.09,72.88,34.02,0.42,Atenção
5,BENEVIDES,2025,0.73,80.72,0.09,88.27,48.34,0.40,Atenção
6,SANTA BÁRBARA DO PARÁ,2025,0.89,65.69,0.00,85.71,3.95,0.38,Atenção
7,ANANINDEUA,2025,2.22,80.72,37.44,72.53,34.02,0.23,Estável



Retorno médio por município (quanto investir afeta a taxa):


,municipio,taxa_media,investimento_medio,cobertura_agua_media,tratamento_esgoto_medio,elasticidade_inv_taxa
0,ANANINDEUA,16.27,2.51,80.72,32.79,-0.84
2,BELÉM,14.24,1.00,80.72,0.09,0.54
7,SANTA IZABEL DO PARÁ,9.67,3.16,84.40,0.09,0.30
5,MARITUBA,8.52,1.70,80.72,0.09,0.60
4,CASTANHAL,7.55,2.33,80.72,0.09,0.50
1,BARCARENA,5.43,4.14,80.72,0.09,0.17
3,BENEVIDES,3.31,5.47,82.89,0.09,-0.21
6,SANTA BÁRBARA DO PARÁ,1.93,3.29,65.57,0.00,0.47



ODS Tracker consolidado (RMB):



,ano,populacao_total,internacoes_hidricas_total,internacoes_total,idx_atend_agua_total_media,idx_tratamento_esgoto_media,pct_conformes_global_media,pct_despesa_investimentos_media,chuva_total_mm_media,taxa_hidricas_10k_rmb,status_taxa_hidricas_10k_rmb,status_pct_conformes_global_media,status_idx_atend_agua_total_media,status_idx_tratamento_esgoto_media
0,2018,2613346.00,5610,137308,79.67,4.75,78.04,2.97,3181.35,21.47,OK,Alerta,Alerta,Alerta
1,2019,2634954.00,5189,132003,79.38,4.75,74.73,3.45,3270.65,19.69,OK,Alerta,Alerta,Alerta
2,2020,2656205.00,3895,122663,79.23,0.09,79.57,3.32,3325.78,14.66,OK,Alerta,Alerta,Alerta
3,2021,2677089.00,2986,130364,79.34,4.75,76.88,3.05,2798.25,11.15,OK,Alerta,Alerta,Alerta
4,2022,2638631.00,1731,130994,80.13,4.76,83.56,3.24,2567.50,6.56,OK,Alerta,Alerta,Alerta
5,2023,2638631.00,1332,139641,79.57,4.75,74.75,1.67,2024.42,5.05,OK,Alerta,Alerta,Alerta
6,2024,2539097.00,1640,153519,79.57,4.75,74.04,3.01,2822.17,6.46,OK,Alerta,Alerta,Alerta
7,2025,2544868.00,980,103581,79.57,4.75,76.65,2.90,2440.03,3.85,OK,Alerta,Alerta,Alerta


## 6. Exportação dos datasets para o Looker Studio
Geramos `painel_prioridade.csv` (ranking + categoria Estável/Atenção/Crítico) e `ods_tracker.csv` (metas ODS 6/3/11) com timestamp UTC. Esses arquivos abastecem diretamente o Painel Prioridade e o ODS Tracker no Looker, garantindo versionamento dentro do repositório.

In [19]:
from datetime import datetime, timezone

export_timestamp = datetime.now(timezone.utc).isoformat()
painel_cols = [
    "cod_mun",
    "municipio",
    "ano",
    "populacao",
    "internacoes_hidricas",
    "internacoes_hidricas_10k",
    "internacoes_total_10k",
    "idx_atend_agua_total",
    "idx_tratamento_esgoto",
    "idx_coleta_esgoto",
    "pct_conformes_global",
    "idx_perdas_distribuicao",
    "despesa_saude_pc",
    "pct_despesa_investimentos_saude",
    "chuva_total_mm",
    "temp_media_c",
    "deficit_atendimento",
    "deficit_tratamento",
    "alerta_qualidade",
    "perdas_excesso",
]

painel_prioridade_export = (
    analysis_df[painel_cols + ["score_priorizacao", "prioridade_categoria"]]
    .sort_values(["ano", "score_priorizacao"], ascending=[False, False])
    .reset_index(drop=True)
    .assign(gerado_em=export_timestamp)
)

painel_prioridade_path = DASHBOARD_EXPORT_DIR / "painel_prioridade.csv"
painel_prioridade_export.to_csv(painel_prioridade_path, index=False)

ods_tracker_export = ods_tracker.copy().assign(gerado_em=export_timestamp)
ods_tracker_path = DASHBOARD_EXPORT_DIR / "ods_tracker.csv"
ods_tracker_export.to_csv(ods_tracker_path, index=False)

print("Arquivos para o Looker atualizados:")
print(f"  • {painel_prioridade_path.relative_to(PROJECT_ROOT)}")
print(f"  • {ods_tracker_path.relative_to(PROJECT_ROOT)}")

print("\nPrévia do Painel Prioridade:")
display(painel_prioridade_export.head(10))

print("\nPrévia do ODS Tracker:")
display(ods_tracker_export.tail())

Arquivos para o Looker atualizados:
  • dashboard/material_para_dashboard/painel_prioridade.csv
  • dashboard/material_para_dashboard/ods_tracker.csv

Prévia do Painel Prioridade:


,cod_mun,municipio,ano,populacao,internacoes_hidricas,internacoes_hidricas_10k,internacoes_total_10k,idx_atend_agua_total,idx_tratamento_esgoto,idx_coleta_esgoto,...,pct_despesa_investimentos_saude,chuva_total_mm,temp_media_c,deficit_atendimento,deficit_tratamento,alerta_qualidade,perdas_excesso,score_priorizacao,prioridade_categoria,gerado_em
0,1501303,BARCARENA,2025,139076.00,61,4.39,427.03,80.72,0.09,1.10,...,3.01,1059.80,25.82,19.28,99.91,32.27,34.02,0.47,Atenção,2025-11-19T17:25:43.616823+00:00
1,1502400,CASTANHAL,2025,209126.00,130,6.22,529.97,80.72,0.09,1.10,...,2.18,2551.60,26.43,19.28,99.91,24.18,34.02,0.47,Atenção,2025-11-19T17:25:43.616823+00:00
2,1501402,BELÉM,2025,1397315.00,605,4.33,418.14,80.72,0.09,1.10,...,0.70,2701.40,26.96,19.28,99.91,25.54,34.02,0.45,Atenção,2025-11-19T17:25:43.616823+00:00
3,1506500,SANTA IZABEL DO PARÁ,2025,79195.00,52,6.57,413.79,86.55,0.09,1.10,...,3.49,2551.60,26.43,13.45,99.91,24.18,34.02,0.45,Atenção,2025-11-19T17:25:43.616823+00:00
4,1504422,MARITUBA,2025,119437.00,12,1.00,308.20,80.72,0.09,1.10,...,1.82,2701.40,26.96,19.28,99.91,27.12,34.02,0.42,Atenção,2025-11-19T17:25:43.616823+00:00
5,1501501,BENEVIDES,2025,68962.00,5,0.73,332.36,80.72,0.09,1.10,...,6.08,2701.40,26.96,19.28,99.91,11.73,48.34,0.40,Atenção,2025-11-19T17:25:43.616823+00:00
6,1506351,SANTA BÁRBARA DO PARÁ,2025,22530.00,2,0.89,290.28,65.69,0.00,1.10,...,3.35,2551.60,26.43,34.31,100.00,14.29,3.95,0.38,Atenção,2025-11-19T17:25:43.616823+00:00
7,1500800,ANANINDEUA,2025,509227.00,113,2.22,357.95,80.72,37.44,1.10,...,2.53,2701.40,26.96,19.28,62.56,27.47,34.02,0.23,Estável,2025-11-19T17:25:43.616823+00:00
8,1506500,SANTA IZABEL DO PARÁ,2024,78317.00,135,17.24,722.70,86.55,0.09,1.10,...,4.59,2641.60,27.42,13.45,99.91,24.18,34.02,0.57,Atenção,2025-11-19T17:25:43.616823+00:00
9,1501402,BELÉM,2024,1398531.00,1078,7.71,610.88,80.72,0.09,1.10,...,0.30,3184.20,27.88,19.28,99.91,32.24,34.02,0.51,Atenção,2025-11-19T17:25:43.616823+00:00



Prévia do ODS Tracker:


,ano,populacao_total,internacoes_hidricas_total,internacoes_total,idx_atend_agua_total_media,idx_tratamento_esgoto_media,pct_conformes_global_media,pct_despesa_investimentos_media,chuva_total_mm_media,taxa_hidricas_10k_rmb,status_taxa_hidricas_10k_rmb,status_pct_conformes_global_media,status_idx_atend_agua_total_media,status_idx_tratamento_esgoto_media,gerado_em
3,2021,2677089.00,2986,130364,79.34,4.75,76.88,3.05,2798.25,11.15,OK,Alerta,Alerta,Alerta,2025-11-19T17:25:43.616823+00:00
4,2022,2638631.00,1731,130994,80.13,4.76,83.56,3.24,2567.50,6.56,OK,Alerta,Alerta,Alerta,2025-11-19T17:25:43.616823+00:00
5,2023,2638631.00,1332,139641,79.57,4.75,74.75,1.67,2024.42,5.05,OK,Alerta,Alerta,Alerta,2025-11-19T17:25:43.616823+00:00
6,2024,2539097.00,1640,153519,79.57,4.75,74.04,3.01,2822.17,6.46,OK,Alerta,Alerta,Alerta,2025-11-19T17:25:43.616823+00:00
7,2025,2544868.00,980,103581,79.57,4.75,76.65,2.90,2440.03,3.85,OK,Alerta,Alerta,Alerta,2025-11-19T17:25:43.616823+00:00
